In [1]:
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
def get_matr(R):
    
    """helper function for the target function"""
    return np.c_[
        R,
        np.pad(R[:-1],(1,0),mode='constant'),
        np.pad(R[:-2],(2,0),mode='constant')
    ].T

In [3]:
costs = pd.DataFrame({
    'week_1': [4,5,7,1,5],
    'week_2': [6,2,1,3,3],
    'week_3': [3,5,1,6,4]
})
constrains = pd.DataFrame({
    'early_start': [1,1,2,2,3],
    'late_start': [4,3,5,6,5]
})

In [4]:
f_target = lambda X: np.c_[[np.dot(costs.loc[i],get_matr(X[i])) for i in range(num_mechs)]].sum(axis=0).max()

In [5]:
num_weeks = 8
num_mechs = 5
X = np.zeros((num_mechs,num_weeks),dtype='uint8')

Xvar = np.zeros((num_mechs,num_weeks,2),dtype='uint8') # all X variants which we will use for filtering
Xvar[:,:,1] = 1

In [6]:
A = np.arange(1,X.shape[1]+1)

In [7]:
pd.concat([costs,constrains],axis=1)

,week_1,week_2,week_3,early_start,late_start
0,4,6,3,1,4
1,5,2,5,1,3
2,7,1,1,2,5
3,1,3,6,2,6
4,5,3,4,3,5


In [8]:
def check_constraints(X):
    """helper function to check if X fits the constraints"""
    
    week_nums = X.dot(np.arange(1,X.shape[1]+1)) #calculates a row of actual start week numbers
    bool_check = (X.sum(axis=1) == 1).all()
    early_check = (week_nums >= constrains['early_start']).all()
    late_check = (week_nums <= constrains['late_start']).all()
    if not bool_check:
        print('not exactly one week start in a row')
    else: 
        if not early_check:
            print('early bounds unmatched')
        else: 
            if not late_check:
                print('late bounds unmatched')
            else:
                print('all good')

In [9]:
check_constraints(X)

not exactly one week start in a row


In [10]:
f_target(X)

0

first we shall core the procedure that will perform $W_1$ filtering by constraints

In [11]:
i = 0 #rows
j = 0 #columns

In [12]:
def Ax(Xvar,i,j,mode=np.max):
    return A[:j].dot(mode(Xvar[i,:j],axis=1))+A[j+1:].dot(mode(Xvar[i,j+1:],axis=1))
def Cx(Xvar,i,j,mode=np.max):
    return mode(Xvar[i,:j],axis=1).sum() + mode(Xvar[i,j+1:],axis=1).sum()

In [13]:
b1 = A[j]*Xvar[i,j] > constrains['late_start'][i] - Ax(Xvar,i,j,np.min)

b2 = A[j]*Xvar[i,j] < constrains['early_start'][i] - Ax(Xvar,i,j,np.max)

b3 = Xvar[i,j] > 1 - Cx(Xvar,i,j,np.min)

b4 = Xvar[i,j] < 1 - Cx(Xvar,i,j,np.max)

In [14]:
np.r_[[b1,b2,b3,b4]].any(axis=0)

array([False, False], dtype=bool)

In [15]:
def check_W1(Xvar,i,j):
    b1 = A[j]*Xvar[i,j] > constrains['late_start'][i] - Ax(Xvar,i,j,np.min)
    b2 = A[j]*Xvar[i,j] < constrains['early_start'][i] - Ax(Xvar,i,j,np.max)
    b3 = Xvar[i,j] > 1 - Cx(Xvar,i,j,np.min)
    b4 = Xvar[i,j] < 1 - Cx(Xvar,i,j,np.max)
    return np.r_[[b1,b2,b3,b4]].any(axis=0) # if true by one axis - filter this value

In [16]:
check_W1(Xvar,0,4)

array([False,  True], dtype=bool)

In [17]:
def filter_W1(Xvar):
    for i in range(Xvar.shape[0]):
        for j in range(Xvar.shape[1]):
            res = check_W1(Xvar,i,j)
            if res.any():
                print(i,j)

In [18]:
filter_W1(Xvar)

0 4
0 5
0 6
0 7
1 3
1 4
1 5
1 6
1 7
2 5
2 6
2 7
3 6
3 7
4 5
4 6
4 7
